In [ ]:
# Import system modules
import arcpy
from arcpy.sa import *
from arcpy.ia import *

In [ ]:
# Check out the ArcGIS Spatial Analyst extension license
arcpy.CheckOutExtension("Spatial")

In [ ]:
# Set the analysis environments
arcpy.env.workspace = "C:/Users/sdion/Desktop/Geospatial Programming/Geospatial_Programming_Project.gdb"
# Check out the ArcGIS Image Analyst extension license
arcpy.CheckOutExtension("ImageAnalyst")

In [ ]:
arcpy.env.overwriteOutput = True
print(arcpy.env.workspace)

In [ ]:
# Import tif, shapefiles and csv files into the geodatabase
from arcpy import env
arcpy.conversion.RasterToGeodatabase("et0_v3_yr.tif;wc2.1_30s_bio_12.tif", 
                                     "Geospatial_Programming_Project.gdb", "")
arcpy.management.XYTableToPoint("Combined_Stations.csv", "Geospatial_Programming_Project.gdb/Weather_Stations", "Longtitude", "Latitude", "#", arcpy.SpatialReference("WGS 1984"))
arcpy.conversion.FeatureClassToFeatureClass("USA_Counties.shp",
                                           "Geospatial_Programming_Project.gdb",
                                           "USA_Counties")

In [ ]:
# Input DEM raster file (TIFF format) & Set local variables
Annual_ET0 = "et0_v3_yr.tif"
Annual_Prec = "wc2.1_30s_bio_12.tif"
USA_Counties = "USA_Counties.shp"

In [ ]:
# Define input and output paths
input_table = "Geospatial_Programming_Project.gdb/Weather_Stations"
output_table = "Geospatial_Programming_Project.gdb/stations_utm"

In [ ]:
# Define the output coordinate system (NAD 1983 UTM Zone 15N)
output_coordinate_system = arcpy.SpatialReference("NAD 1983 UTM Zone 15N")

# Project the feature class to the desired coordinate system
stations_utm = arcpy.management.Project(input_table, output_table, output_coordinate_system)

# You can also print the output variable if you want to confirm its name and location
print(stations_utm)

In [ ]:
arcpy.Describe(stations_utm)

In [ ]:
# output data
counties_utm = "USA_Counties_utm.shp"

# create a spatial reference object for the output coordinate system
out_coordinate_system = arcpy.SpatialReference('NAD 1983 UTM Zone 15N')

# run the tool
arcpy.Project_management(USA_Counties, counties_utm, out_coordinate_system)

In [ ]:
arcpy.Describe(Annual_ET0)

arcpy.Describe(Annual_Prec)

In [ ]:
# Define the input raster and output raster paths
input_raster = Annual_ET0
output_raster = "Annual_ET0_utm"

# Remove invalid characters from the output raster name
output_raster = arcpy.ValidateTableName(output_raster)

# Define the spatial reference using a factory code
spatial_ref = arcpy.SpatialReference(26915)  # Factory code for NAD 1983 UTM Zone 15N

# Project the raster
arcpy.ProjectRaster_management(input_raster, output_raster, spatial_ref)

In [ ]:
# Define the input raster and output raster paths
input_raster1 = Annual_Prec
output_raster1 = "Annual_Prec_utm"

# Remove invalid characters from the output raster name
output_raster = arcpy.ValidateTableName(output_raster)

# Define the spatial reference using a factory code
spatial_ref = arcpy.SpatialReference(26915)  # Factory code for NAD 1983 UTM Zone 15N

# Project the raster
arcpy.ProjectRaster_management(input_raster1, output_raster1, spatial_ref)

In [ ]:
# Select Perry county in Missouri by using the FIPS code and the name of the county
sql = "FIPS = '29157'"
perry_lyr = arcpy.management.SelectLayerByAttribute(counties_utm, "NEW_SELECTION",sql)

# Write the selected features to a new feature class
arcpy.management.CopyFeatures(perry_lyr, 'perry')

In [ ]:
# Set input and output raster paths
output_raster2 = "AnET0"

# Clip the raster
arcpy.management.Clip(output_raster, "#", output_raster2, "perry", "#", "ClippingGeometry")

In [ ]:
# Set input and output raster paths
output_raster3 = "AnPrec"

# Clip the raster
arcpy.management.Clip(output_raster1, "#", output_raster3, "perry", "#", "ClippingGeometry")

In [ ]:
# Set local variables
AnET0_Perry = output_raster2
AnPrec_Perry = output_raster3

# Execute Minus
outMinus = Minus(AnPrec_Perry,AnET0_Perry)

# Save the output 
outMinus.save("C:/Users/sdion/Desktop/Geospatial Programming/Geospatial_Programming_Project.gdb/Eff_Rain_Perry.tif")

In [ ]:
import arcpy
from arcpy import env  
from arcpy.sa import *

In [ ]:
# Create points using the provided coordinates
point1 = arcpy.Point(-89.791667, 37.666667)
point2 = arcpy.Point(-89.875, 37.75)

# Create point geometries from the points
point_geom1 = arcpy.PointGeometry(point1, arcpy.SpatialReference(4326))  # WGS 1984
point_geom2 = arcpy.PointGeometry(point2, arcpy.SpatialReference(4326))  # WGS 1984

# Define the output coordinate system (NAD 1983 UTM Zone 15N)
output_coordinate_system = arcpy.SpatialReference("NAD 1983 UTM Zone 15N")

# Project the point geometries to the desired coordinate system
projected_point1 = point_geom1.projectAs(output_coordinate_system)
projected_point2 = point_geom2.projectAs(output_coordinate_system)

# Print the projected coordinates
print("Projected coordinates of (-89.791667, 37.666667): ", projected_point1.centroid)
print("Projected coordinates of (-89.875, 37.75): ", projected_point2.centroid)

In [ ]:
# Set the extent environment
arcpy.env.extent = arcpy.Extent(782991.06, 4173676.54, 775329.93, 4182676.98)

# Perform IDW interpolation
outIDW = arcpy.sa.Idw("Geospatial_Programming_Project.gdb/stations_utm", "Annual_Effective_Rainfall_mm_2023", 17, 1)

# Save the result
outIDW.save("Geospatial_Programming_Project.gdb/Idw_Project.shp")

In [ ]:
# Execute Minus to find the difference between the historic and modern effective rainfall
Eff_Rain_diff = Minus(outMinus,outIDW)

# Save the output 
outMinus.save("C:/Users/sdion/Desktop/Geospatial Programming/Geospatial_Programming_Project.gdb/Eff_Rain_diff.tif")